In [1]:
from __future__ import print_function    # (at top of module)
import sys
import time

import json
import requests
from props.default import *


# Miscellaneous Functions

In [2]:
def log(m):
        print(m)
        
def setToken(t):
    global DL_TOKEN 
    DL_TOKEN = { 'Authorization' : 'DirectLogin token=%s' % t}
    
def mergeHeaders(x, y):
    z = x.copy()
    z.update(y)
    return z

def merge_headers_list(header_list):
    output_dict = {}
    for header in header_list:
        output_dict.update(header)
    return output_dict
        

CONTENT_JSON  = { 'content-type'  : 'application/json' }

# 1. Login in

In [3]:
def login(username, password, consumer_key):
    login_url = '{0}/my/logins/direct'.format(BASE_URL)
    login_header  = { 'Authorization' : 'DirectLogin username="%s",password="%s",consumer_key="%s"' % (username, password, consumer_key)}
    # Login and receive authorized token
    log('Login as {0} to {1}'.format(login_header, login_url))
    r = requests.post(login_url, headers=login_header)
    if (r.status_code != 201):
        log("error: could not login")
        log("text: " + r.text)
        return r.text
    t = r.json()['token']
    log("Received token: {0}".format(t))
    setToken(t)
    return t, login_header

In [4]:
token, login_header = login('username', 'mdp', 'api_key')


Login as {'Authorization': 'DirectLogin username="tkarch",password="85xdduz6xvmumP!",consumer_key="2limvjs5zbsa5lkajq4vdmtqmtyyb4sgo12j1vap"'} to https://bnpparibas-irb.openbankproject.com/my/logins/direct
Received token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyIiOiIifQ.k-m0yeDT5uIz_R-Mu51YfSQX87_ziznu-HmW3Xjg940


# 2. Data Warehouse

To have access to the datawarehouse you have to request the role on:
https://bnpparibas-irb-explorer.openbankproject.com/#vv3_0_0-dataWarehouseSearch

Here is the list of index of the elastic search DB:
    
    
irb-declared-products, irb-sme-transactions, irb-transactions, irb-contracts, irb-taxes, irb-charges, irb-revenues, irb-individual-clients, irb-markets, irb-sme-client, irb-assets, irb-sme-contracts
bow-declared-products, bow-sme-transactions, bow-transactions, bow-contracts, bow-taxes, bow-charges, bow-revenues, bow-individual-clients, bow-markets, bow-sme-client, bow-assets, bow-sme-contracts
20170608-markets, 20170608-taxes, 20170608-charges


In [5]:
def request_datawarehouse_body(field, first_request_bool, last_index=None):
    post_data = {
        "size" : 10000,
        "query" : {"match_all":{}},
        "search_after" : [last_index],
        "sort" : [{"customer_id":"asc"}]
    }
    
    post_data_first_request = {
        "size" : 10000,
        "query" : {"match_all":{}},
        "sort" : [{"customer_id":"asc"}]
    }
    if first_request_bool:
        post_data=post_data_first_request
        
    response = requests.post(u"{0}/obp/{1}/search/warehouse/{2}".format(BASE_URL, API_VERSION, field), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()

In [8]:
def request_datawarehouse(field, n):
    results = []
    first_request = True
    last_index = None
    
    while len(results) < n:
        print('iter')
        result = request_datawarehouse_body(field, first_request, last_index)
        first_request = False
        print(result['json'])
        last_index = result['json']['hits']['hits'][-1]['_source']['customer_id']
        results.extend(result['json']['hits']['hits'])
    return results

In [9]:
request_datawarehouse('irb-contracts', 1)

iter


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


[{'_id': '421699',
  '_index': 'irb-contracts',
  '_score': None,
  '_source': {'balance': '0',
   'balance_date': '2017-04-05',
   'contract_id': '379154',
   'customer_id': '1',
   'date_opened': '2012-02-07',
   'product_category': 'LIFE INSURANCE CONTRACTS',
   'product_family': 'HEALTH AND PENSION FUND CONTRACT',
   'product_id': '09ASPRES',
   'product_name': 'ASSURCOMPTE',
   'product_supfamily': 'INSURANCE'},
  '_type': 'default',
  'sort': ['1']},
 {'_id': '668771',
  '_index': 'irb-contracts',
  '_score': None,
  '_source': {'balance': '0',
   'balance_date': '2017-04-05',
   'contract_id': '720971',
   'customer_id': '1',
   'date_opened': '2006-01-05',
   'product_category': 'OTHER RETIREMENT FUND CONTRACTS',
   'product_family': 'PROTECTION ASSISTANCE',
   'product_id': '09SEC-PL',
   'product_name': 'BNP SECURITY PLUS',
   'product_supfamily': 'INSURANCE'},
  '_type': 'default',
  'sort': ['1']},
 {'_id': '514584',
  '_index': 'irb-contracts',
  '_score': None,
  '_source

In [ ]:
def request_datawarehouse_agg():
    post_data = {
         "size" : 20,
        "aggs": {
            "byindex": {
                  "terms": {
                    "field": "_index"
                  }
                }
        },
        "query"   : {"match_all":{          }  },
    }
    # Send post request with attached json
    response = requests.post(u"{0}/obp/{1}/search/warehouse/{2}".format(BASE_URL, API_VERSION, 'ALL'), json=post_data, headers=mergeHeaders(DL_TOKEN, CONTENT_JSON))
    return response.json()